Import thư viện cần thiết

In [1]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn import linear_model
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import matplotlib.pyplot as plt
from importlib import reload
from xgboost import XGBClassifier
plt=reload(plt)

ModuleNotFoundError: No module named 'xgboost'

Đọc dữ liệu dataset

In [ ]:
train_df = pd.read_csv('train dataset.csv')
test_df = pd.read_csv('test dataset.csv')

: 

Đổi tên y của test_df và train_df

In [ ]:
train_df.rename(columns = {'Personality (Class label)':'Personality'}, inplace = True)
test_df.rename(columns = {'Personality (class label)':'Personality'}, inplace = True)

: 

In [ ]:
train_df.head()

: 

Trực quan hoá các đặc điểm đầu vào của train_df

In [ ]:
sns.boxplot(x=train_df['Gender'], y=train_df['Personality'])
plt.xlabel('Giới tính')
plt.ylabel('Tính cách')
plt.yticks([0,1,2,3,4] , ['Trách nhiệm', 'Sôi nổi', 'Đáng tin cậy', 'Nghiêm túc', 'Hướng ngoại'])
# plt.xticks([0, 1], ['Nam', 'Nữ'])
plt.show()

: 

In [ ]:
sns.boxplot(x = train_df['Personality'], y = train_df['Age'])
plt.ylabel("Tuổi")
plt.xlabel('Tính cách')
plt.xticks([0,1,2,3,4] , ['Hướng ngoại', 'Nghiêm túc', 'Đáng tin cậy', 'Sôi nổi', 'Trách nhiệm'])
plt.show()

: 

In [ ]:
filtered_df = train_df[(train_df['Gender'] != 'Male') & (train_df['Gender'] != 'Female')]

# Hiển thị 1 dòng từ filtered_df
print(filtered_df.head(1).rename(columns={
    "Gender": "Giới tính",
    "Age": "Tuổi",
    "openness": "Hoà đồng",
    "neuroticism": "Nhạy cảm",
    "conscientiousness": "Tận tâm",
    "agreeableness": "Hoà thuận",
    "extraversion": "Hướng ngoại",
    "Personality": "Tính cách"
}))

: 

In [ ]:
sns.boxplot(x = train_df['Personality'], y = train_df['conscientiousness'])
plt.xlabel('Tính cách')
plt.xticks([0,1,2,3,4] , ['Hướng ngoại', 'Nghiêm túc', 'Đáng tin cậy', 'Sôi nổi', 'Trách nhiệm'])
plt.ylabel('Tận tâm')
plt.show()

: 

In [ ]:
sns.boxplot(x = train_df['Personality'], y = train_df['extraversion'])
plt.xlabel('Tính cách')
plt.xticks([0,1,2,3,4] , ['Hướng ngoại', 'Nghiêm túc', 'Đáng tin cậy', 'Sôi nổi', 'Trách nhiệm'])
plt.ylabel('Hướng ngoại')
plt.show()

: 

In [ ]:
sns.boxplot(x = train_df['Personality'], y = train_df['neuroticism'])
plt.xlabel('Tính cách')
plt.xticks([0,1,2,3,4] , ['Hướng ngoại', 'Nghiêm túc', 'Đáng tin cậy', 'Sôi nổi', 'Trách nhiệm'])
plt.ylabel('Nhạy cảm')
plt.show()

: 

In [ ]:
sns.boxplot(x = train_df['Personality'], y = train_df['agreeableness'])
plt.xlabel('Tính cách')
plt.xticks([0,1,2,3,4] , ['Hướng ngoại', 'Nghiêm túc', 'Đáng tin cậy', 'Sôi nổi', 'Trách nhiệm'])
plt.ylabel('Hoà thuận')
plt.show()

: 

In [ ]:
sns.boxplot(x = train_df['Personality'], y = train_df['openness'])
plt.xlabel('Tính cách')
plt.xticks([0,1,2,3,4] , ['Hướng ngoại', 'Nghiêm túc', 'Đáng tin cậy', 'Sôi nổi', 'Trách nhiệm'])
plt.ylabel('Hoà đồng')
plt.show()

: 

Tiền xử lý dữ liệu: gender thành dạng số: 1 nếu là nam, 0 nếu là nữ hoặc dữ liệu không nằm trong nam và nữ

In [ ]:
# train_df = train_df[train_df['Gender'].isin(['Male', 'Female'])]

: 

In [ ]:
train_df['Gender'] = train_df['Gender'].apply(lambda x: 1 if x == "Male" else 0)
test_df['Gender'] = test_df['Gender'].apply(lambda x: 1 if x == "Male" else 0)
train_df.head()

train_df["Personality"] = train_df["Personality"].replace(
                                                                {"extraverted":0,
                                                                "dependable":1,
                                                                "lively":2,
                                                                "responsible":3,
                                                                "serious":4})
test_df["Personality"] = test_df["Personality"].replace(
                                                                {"extraverted":0,
                                                                "dependable":1,
                                                                "lively":2,
                                                                "responsible":3,
                                                                "serious":4})
train_df_translate = train_df.rename(columns={
    "Gender": "Giới tính",
    "Age": "Tuổi",
    "openness": "Mở lòng",
    "neuroticism": "Cảm xúc",
    "conscientiousness": "Tận tụy",
    "agreeableness": "Dễ tính",
    "extraversion": "Hướng ngoại",
    "Personality": "Tính cách"
})

train_df.head().rename(columns={
    "Gender": "Giới tính",
    "Age": "Tuổi",
    "openness": "Hoà đồng",
    "neuroticism": "Nhạy cảm",
    "conscientiousness": "Tận tâm",
    "agreeableness": "Hoà thuận",
    "extraversion": "Hướng ngoại",
    "Personality": "Tính cách"
})

: 

In [ ]:
# Cài đặt mô hình SVM với kernel linear, C = 5
# model = SVC(kernel='linear', C=5)
# model = linear_model.LogisticRegression(multi_class='multinomial', solver='newton-cg',max_iter =1000)
model = XGBClassifier(n_estimators = 750, booster = "gblinear")
x_train = train_df.drop('Personality', axis=1).values
y_train = train_df['Personality'].values

x_test = test_df.drop('Personality', axis=1).values
y_test = test_df['Personality'].values
# Huấn luyện mô hình trên tập dữ liệu huấn luyện
model.fit(x_train, y_train)

#dump model
# joblib.dump(model, "SVC.joblib")
joblib.dump(model, "XGBoost.joblib")

: 

In [ ]:
# Đánh giá mô hình trên tập dữ liệu kiểm tra
y_pred = model.predict(x_test)

# Tính độ chính xác của mô hình
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# In báo cáo phân loại
print(classification_report(y_test, y_pred))

: 

In [ ]:
# Đánh giá mô hình bằng confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Vẽ biểu đồ cho confusion matrix
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

print("Classification Report:")
print(classification_report(y_test, y_pred))


: 

In [ ]:
# Mapping dictionary
label_map = {0: "extraverted", 1: "dependable", 2: "lively", 3: "responsible", 4: "serious"}

# Applying mapping using indexing
y_pred_labels = np.array([label_map[val] for val in y_pred])

print(y_pred_labels)

: 